# Supply Chain Data Cleaning
#### Objective: Clean the dirty dataset to prepare for reliable analysis

In [89]:
# Imports
import pandas as pd
import numpy as np

In [90]:
# Import Findings from first document
with open("C:/Users/User/OneDrive/DS/Projekte/SCM Analysis/data/explanations/key_findings_exploration.md", "r") as f:
    content = f.read()
    print(content)


### Summary of Key Findings for Data Cleaning

1. **Missing Values**
- Shipping costs: 5 missing values
- Customer demographics: 6 missing values

2. **Data Type Issues**
- Availability appears to be stored as string
- Needs conversion to integer

3. **Duplicate Rows**
- 1 duplicated row detected

4. **Outliers**
- Shipping times contains a negative value (-5)
- Defect rates includes an extreme value of 100

5. **Inconsistent Categories**
- Transportation modes: lowercase values (e.g. "road")
- Shipping carriers: extra spaces (e.g. "DHL ")

6. **Potential Redundant Columns**
- Lead time and Lead times have the same number of unique values – further check needed



In [91]:
# Load the raw (dirty) dataset
df = pd.read_csv("C:/Users/User/OneDrive/DS/Projekte/SCM Analysis/data/supply_chain_data.csv")

1. Missing Values

In [92]:
# 1. Check for missing values -> assumption from the first document
print("Missing values per column:")
print(df.isnull().sum())

Missing values per column:
Product type               0
SKU                        0
Price                      0
Availability               0
Number of products sold    0
Revenue generated          0
Customer demographics      6
Stock levels               0
Lead times                 0
Order quantities           0
Shipping times             0
Shipping carriers          0
Shipping costs             5
Supplier name              0
Location                   0
Lead time                  0
Production volumes         0
Manufacturing lead time    0
Manufacturing costs        0
Inspection results         0
Defect rates               0
Transportation modes       0
Routes                     0
Costs                      0
dtype: int64


- We used the median to fill missing shipping costs because it is not effected by extreme values. Mean could be pulled up by very high shipping prices, but the median stays more stable.
- We filled missing customer demographics with "Unknown" because we don’t had enough information to guess the real value. It’s better to keep the data than remove the whole row.

In [93]:
# Fill missing values in 'Shipping costs' with the median
median_shipping_cost = df['Shipping costs'].median()
df['Shipping costs'] = df['Shipping costs'].fillna(median_shipping_cost)


In [94]:
# Fill missing values in 'Customer demographics' with 'Unknown'
df['Customer demographics'] = df['Customer demographics'].fillna("Unknown")

In [95]:
# Check again for missing values -> assumption from the first document
print("Missing values per column:")
print(df.isnull().sum())

Missing values per column:
Product type               0
SKU                        0
Price                      0
Availability               0
Number of products sold    0
Revenue generated          0
Customer demographics      0
Stock levels               0
Lead times                 0
Order quantities           0
Shipping times             0
Shipping carriers          0
Shipping costs             0
Supplier name              0
Location                   0
Lead time                  0
Production volumes         0
Manufacturing lead time    0
Manufacturing costs        0
Inspection results         0
Defect rates               0
Transportation modes       0
Routes                     0
Costs                      0
dtype: int64


2. Duplicated rows

In [96]:
# 2. Remove duplicate rows
duplicates = df.duplicated().sum()
print(f"Duplicate rows found: {duplicates}")
df.drop_duplicates(inplace=True)

Duplicate rows found: 1


3. Data Types


In [97]:
# 3. Fix data types
# 'Availability' might be object due to dirty data – try converting it to numeric
df['Availability'] = pd.to_numeric(df['Availability'], errors='coerce')

In [98]:
# After coercion, check again for nulls
df['Availability'] = df['Availability'].fillna(df['Availability'].median())
df['Availability'] = df['Availability'].astype(int)

4. Categorical Text Columns

In [99]:
# 4. Standardize categorical text columns
# Remove trailing spaces and unify formatting in some key columns
df['Shipping carriers'] = df['Shipping carriers'].str.strip()
df['Transportation modes'] = df['Transportation modes'].str.strip().str.capitalize()

5. Outlier handling

In [100]:
# 5. Outlier handling
# Remove negative values in 'Shipping times'
df = df[df['Shipping times'] >= 0]

In [101]:
# Cap extreme defect rates at 99th percentile
q_high = df['Defect rates'].quantile(0.99)
df['Defect rates'] = np.where(df['Defect rates'] > q_high, q_high, df['Defect rates'])

6. Duplicated Columns

In [102]:
# 6. Check for duplicate columns or suspicious overlap
# Example: Check if 'Lead time' and 'Lead times' are the same
if df['Lead time'].equals(df['Lead times']):
    print("Columns 'Lead time' and 'Lead times' are identical.")
    df.drop(columns=['Lead times'], inplace=True)
else:
    print("Columns 'Lead time' and 'Lead times' differ review needed.")

Columns 'Lead time' and 'Lead times' differ review needed.


7. Final Safety Check

In [103]:
# Final sanity check
print("\nCleaned DataFrame info:")
print(df.info())
print("\nRemaining missing values:")
print(df.isnull().sum())



Cleaned DataFrame info:
<class 'pandas.core.frame.DataFrame'>
Index: 99 entries, 0 to 99
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product type             99 non-null     object 
 1   SKU                      99 non-null     object 
 2   Price                    99 non-null     float64
 3   Availability             99 non-null     int32  
 4   Number of products sold  99 non-null     int64  
 5   Revenue generated        99 non-null     float64
 6   Customer demographics    99 non-null     object 
 7   Stock levels             99 non-null     int64  
 8   Lead times               99 non-null     int64  
 9   Order quantities         99 non-null     int64  
 10  Shipping times           99 non-null     int64  
 11  Shipping carriers        99 non-null     object 
 12  Shipping costs           99 non-null     float64
 13  Supplier name            99 non-null     object 
 14  Location

In [104]:
# Save the cleaned dataset
# df.to_csv("../data/supply_chain_data_cleaned.csv", index=False)
# print("\n Cleaned dataset saved successfully.")